## Описание проекта

Федеральный оператор сотовой связи «Мегалайн» предлагает клиентам два тарифных плана: «Смарт» и «Ультра».
Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.

## Цель исследования

Осуществить предварительный анализ тарифов с целью проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

## Описание тарифов

#### Тариф «Смарт»

    Ежемесячная плата: 550 рублей
    Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
    Стоимость услуг сверх тарифного пакета: 1. минута разговора: 3 рубля («Мегалайн» всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута); 2. сообщение: 3 рубля; 3. 1 Гб интернет-трафика: 200 рублей.

#### Тариф «Ультра»

    Ежемесячная плата: 1950 рублей
    Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
    Стоимость услуг сверх тарифного пакета: 1. минута разговора: 1 рубль; 2. сообщение: 1 рубль; 3. 1 Гб интернет-трафика: 150 рублей.

#### Примечание:
«Мегалайн» всегда округляет секунды до минут, а мегабайты — до гигабайт. Каждый звонок округляется отдельно: даже если он длился всего 1 секунду, будет засчитан как 1 минута.
Для веб-трафика отдельные сессии не считаются. Вместо этого общая сумма за месяц округляется в бо́льшую сторону. Если абонент использует 1025 мегабайт в этом месяце, с него возьмут плату за 2 гигабайта.

Описание данных
Таблица users (информация о пользователях):

    user_id — уникальный идентификатор пользователя
    first_name — имя пользователя
    last_name — фамилия пользователя
    age — возраст пользователя (годы)
    reg_date — дата подключения тарифа (день, месяц, год)
    churn_date — дата прекращения пользования тарифом (если значение пропущено, что тариф ещё действовал на момент выгрузки данных)
    city — город проживания пользователя
    tarif — название тарифного плана

Таблица calls (информация о звонках):

    id — уникальный номер звонка
    call_date — дата звонка
    duration — длительность звонка в минутах
    user_id — идентификатор пользователя, сделавшего звонок

Таблица messages (информация о сообщениях):

    id — уникальный номер сообщения
    message_date — дата сообщения
    user_id — идентификатор пользователя, отправившего сообщение

Таблица internet (информация об интернет-сессиях):

    id — уникальный номер сессии
    mb_used — объём потраченного за сессию интернет-трафика (в мегабайтах)
    session_date — дата интернет-сессии
    user_id — идентификатор пользователя

Таблица tariffs (информация о тарифах):

    tariff_name — название тарифа
    rub_monthly_fee — ежемесячная абонентская плата в рублях
    minutes_included — количество минут разговора в месяц, включённых в абонентскую плату
    messages_included — количество сообщений в месяц, включённых в абонентскую плату
    mb_per_month_included — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)
    rub_per_minute — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)
    rub_per_message — стоимость отправки сообщения сверх тарифного пакета
    rub_per_gb — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)

# Шаг 1. Загрузка данных

In [69]:
# импортнем наши любимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from math import ceil
from scipy import stats as st

% matplotlib inline

UsageError: Line magic function `%` not found.


In [70]:
# прочитаем данные из файлов
calls = pd.read_csv('../data/calls.csv')
internet = pd.read_csv('../data/internet.csv')
messages = pd.read_csv('../data/messages.csv')
tariffs = pd.read_csv('../data/tariffs.csv')
users = pd.read_csv('../data/users.csv')

# зададим датафреймам названия, для удобства работы с ними
calls.name = 'calls'
internet.name = 'internet'
messages.name = 'messages'
tariffs.name = 'tariffs'
users.name = 'users'

# создадим список датафреймов, для дальнейшей автоматизации работы с ними
data_list = [calls, internet, messages, tariffs, users]

# словарь для сериализации номеров месяцов в более удобный для представления вид
month_to_str = {
    1: 'Январь',
    2: 'Февраль',
    3: 'Март',
    4: 'Апрель',
    5: 'Май',
    6: 'Июнь',
    7: 'Июль',
    8: 'Август',
    9: 'Сентябрь',
    10: 'Октябрь',
    11: 'Ноябрь',
    12: 'Декабрь'
}

Проверим данные на наличие пропусков

In [71]:
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    # объявляем счетчик
    counter = -1
    display('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        display(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                display(f'  - в колонке {columns[counter]}: {i} пропусков')
    else:
        display(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# применим функцию ко всем датафреймам
for data in data_list:
    get_missing_values(data)

'============================================================'

'Отлично, в датафрейме calls отсутствуют пропуски.'

'============================================================'

'Отлично, в датафрейме internet отсутствуют пропуски.'

'============================================================'

...

'============================================================'

'Отлично, в датафрейме tariffs отсутствуют пропуски.'

'============================================================'

'В датафрейме users имеются следующие пропуски:'

'  - в колонке churn_date: 462 пропусков'

Пропуски есть только в колонке churn_date датафрейма users, что корректно, исходя их описания полученных данных.
Пропуски в этой колонке означают, что тариф ещё действовал на момент выгрузки данных.

## Шаг 2. Предобработка данных

Чутка надоело каждый раз проводить ручное приведение типов, поэтому напишем функцию, которая будет делать это автоматически для int и foat,
а также будет автоматически преобразовывать даты из str в datetime и приводить str в нижний регистр и выводить результат своей работы на экран.

Вопрос к ревьюеру: правда ли нужно приводить int64 и float64 к типам меньшей размерности в EDA?
Конечно приятно сэкономить несколько МБ памяти без черной магии, но я почти уверен, что никто
этим в рабочей деятельности так не заморачивается без конкретной технической необходимости.
А там где big data - используют в том числе и концепцию map/reduce.
Или я не прав?

In [72]:
def auto_change_dtypes(data: pd.DataFrame) -> None:
    """
    Автоматически определяет тип столбца, и изменяет его в соответствии с хранимыми значениями.
    Функция не возвращает новый датафрейм, а изменяет переданный в качестве аргумента.
    Функция заточена под данные конкретного проекта.
    Функция поддерживает автоматическое преобразование следующих типов и форматов данных:
     - int64
     - float64
     - str: если в названии колонки есть date и формат даты %Y-%m-%d,
            то переводит в формат pandas datetime, иначе - переводит строковые данные в нижний регистр

    Пример преобразования:
    data[column] является int64 и содержит значения в диапазоне от 0 до 100 - будет преобразован в int8
    data[date_column] является object и содержит в имени колонки date - будет преобразован в datetime

    :param data: pd.DataFrame
    :return: None
    """
    # получаем количество используемой датафреймом памяти
    memory_usage_before_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    # получаем описание датафрейма
    describe = data.describe()
    # получаем названия колонок
    columns = data.columns.to_list()
    # получаем типы данных
    dtypes = data.dtypes
    # количество типов данных
    indexes = len(dtypes)
    # создаем 2 словаря для int и float, содержащие в качестве ключей типы данных, а значений - список из min и max значений этих типов данных
    correct_int_dtypes = {'int8': [-2**7, 2**7-1], 'int16': [-2**15, 2**15-1], 'int32': [-2**31, 2**31-1]}
    correct_float_dtypes = {'float16': [-2.0**16, 2.0**16-1], 'float32': [-2.0**31, 2.0**31-1]}

    display(f'{"="*30} Работаем с датафреймом {data.name} {"="*30}')
    # пробегаем по индексам колонок датафрейма и типам данных колонок
    for index, dtype in zip(range(0, indexes), dtypes):

        # если тип int64, меняем на тип, соответствующий значениям колонок
        if dtype == np.int64:
            for key, value in zip(correct_int_dtypes.keys(), correct_int_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип float64, меняем на тип, соответствующий значениям колонок
        elif dtype == np.float64:
            for key, value in zip(correct_float_dtypes.keys(), correct_float_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип object и колонка содержит в названии 'date' - меняем на datetime
        elif dtype == object:
            if 'date' in columns[index]:
                display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на datetime')
                data[columns[index]] = pd.to_datetime(data[columns[index]], format='%Y-%m-%d')
            # иначе приводим данные к нижнему регистру
            else:
                data[columns[index]] = data[columns[index]].apply(lambda s: s.lower())

    # количество памяти, используемое датафреймом оптимизации типов данных
    memory_usage_after_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    bytes_in_mb = 2**23
    display(f'Использование памяти датафрейма до сжатия: {(memory_usage_before_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Использование памяти датафрейма после сжатия: {(memory_usage_after_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Сжато: {((memory_usage_before_change_dtypes - memory_usage_after_change_dtypes) / bytes_in_mb):.2f} мб.')

for data in data_list:
    auto_change_dtypes(data)

'============================== Работаем с датафреймом calls =============================='

'Изменяем тип колонки call_date датафрейма calls с object на datetime'

'Изменяем тип колонки duration датафрейма calls с float64 на float16'

'Изменяем тип колонки user_id датафрейма calls с int64 на int16'

'Использование памяти датафрейма до сжатия: 3.57 мб.'

...

'Изменяем тип колонки churn_date датафрейма users с object на datetime'

'Изменяем тип колонки reg_date датафрейма users с object на datetime'

'Использование памяти датафрейма до сжатия: 0.03 мб.'

'Использование памяти датафрейма после сжатия: 0.02 мб.'

'Сжато: 0.01 мб.'

В условии сказано, что пропущенные звонки имеют нулевую продолжительность. Для удобства работы с данными создадим дополнительный столбец в датафрейме calls, в котором будет значение True, если звонок был пропущен.

In [73]:
calls['is_missed'] = calls['duration'].apply(lambda x: True if x == 0 else False)
# проверим, что код отработал верно
calls.head()

,id,call_date,duration,user_id,is_missed
0,1000_0,2018-07-25,0.000000,1000,True
1,1000_1,2018-08-17,0.000000,1000,True
2,1000_2,2018-06-11,2.849609,1000,False
3,1000_3,2018-09-21,13.796875,1000,False
4,1000_4,2018-12-15,5.179688,1000,False


В датафрейме internet присутствует колонка Unnamed: 0, что говорит о том, что файл csv был сохранен вместе с индексами.
При загрузке датафрейма из файла, pandas создает новый индекс, а сохраненный в файле именуется 'Unnamed: 0'.
Исправим это.

In [74]:
internet.drop('Unnamed: 0', axis=1, inplace=True)
# проверим, что исправили ошибку
internet.head()

,id,mb_used,session_date,user_id
0,1000_0,112.9375,2018-11-25,1000
1,1000_1,1053.0000,2018-09-07,1000
2,1000_2,1197.0000,2018-06-25,1000
3,1000_3,550.5000,2018-08-22,1000
4,1000_4,302.5000,2018-09-24,1000


В условии сказано, что продолжительность каждого звонка округляется до минут в большую сторону, даже если он длился всего 1 секунду, будет засчитан как 1 минута.
Создадим столбец в датафрейме calls с округленными в большую сторону значениями продолжительности звонка.

In [75]:
calls['rounded_duration'] = calls['duration'].apply(lambda x: ceil(x))
# проверим, что код отработал верно
calls.head()

,id,call_date,duration,user_id,is_missed,rounded_duration
0,1000_0,2018-07-25,0.000000,1000,True,0
1,1000_1,2018-08-17,0.000000,1000,True,0
2,1000_2,2018-06-11,2.849609,1000,False,3
3,1000_3,2018-09-21,13.796875,1000,False,14
4,1000_4,2018-12-15,5.179688,1000,False,6


Посчитаем количество сделанных звонков и израсходованных минут разговора по месяцам.

In [76]:
# посчитаем количество сделанных звонков (не пропущенных) каждым пользователем по месяцам
calls_data = (
    calls
    .query("is_missed == False")
    .groupby(['user_id', calls['call_date'].dt.month]).count()
    .rename({'id': 'calls_count'}, axis=1)['calls_count']
    .reset_index()
)

# добавим столбец с названиями месяцев
calls_data['month'] = calls_data['call_date'].apply(lambda x: month_to_str.get(x))

# проверим что код отработал верно
calls_data.head()

,user_id,call_date,calls_count,month
0,1000,5,17,Май
1,1000,6,28,Июнь
2,1000,7,41,Июль
3,1000,8,42,Август
4,1000,9,46,Сентябрь


In [78]:
# посчитаем количество минут, израсходованных пользователем в месяц
# используем округленные данные, т.к. оператор всегда округляет секунды до минут в большую сторону
# при этом можем не исключать пропущенные, т.к. там нулевая продолжительность, которая не повлияет на сумму
duration_data = (
    calls
    .groupby(['user_id', calls['call_date'].dt.month])
    .agg({'rounded_duration': 'sum'})
    .reset_index()
)

# добавим столбец с названиями месяцев
duration_data['month'] = duration_data['call_date'].apply(lambda x: month_to_str.get(x))

# проверим что код отработал верно
duration_data.head()

,user_id,call_date,rounded_duration,month
0,1000,5,159,Май
1,1000,6,172,Июнь
2,1000,7,340,Июль
3,1000,8,408,Август
4,1000,9,466,Сентябрь


In [80]:
# посчитаем количество отправленных сообщений по месяцам
messages_data = (
    messages
    .groupby(['user_id', messages['message_date'].dt.month])
    .count()
    .rename({'id': 'messages_count'}, axis=1)['messages_count']
    .reset_index()
)

# добавим столбец с названиями месяцев
messages_data['month'] = messages_data['message_date'].apply(lambda x: month_to_str.get(x))

# проверим что код отработал верно
messages_data.head()

,user_id,message_date,messages_count,month
0,1000,5,22,Май
1,1000,6,60,Июнь
2,1000,7,75,Июль
3,1000,8,81,Август
4,1000,9,57,Сентябрь


In [82]:
# посчитаем объем израсходованного интернет трафика каждым пользователем, по месяцам
internet_traff_data = (
    internet
    .groupby(['user_id', internet['session_date'].dt.month])
    .sum()
    .reset_index()
)

# добавим столбец с названиями месяцев
internet_traff_data['month'] = internet_traff_data['session_date'].apply(lambda x: month_to_str.get(x))

# т.к. для веб-трафика отдельные сессии не считаются,
# а вместо этого оператором общая сумма за месяц округляется в бо́льшую сторону,
# то посчитаем количество ГБ, используемых пользователем и округлим полученное значение соответствующим образом
internet_traff_data['rounded_gb_used'] = (internet_traff_data['mb_used'] / 1024).apply(lambda x: ceil(x))

# проверим, что код отработал верно
internet_traff_data.head()

,user_id,session_date,mb_used,month,rounded_gb_used
0,1000,5,2254.0,Май,3
1,1000,6,23232.0,Июнь,23
2,1000,7,14008.0,Июль,14
3,1000,8,14056.0,Август,14
4,1000,9,14568.0,Сентябрь,15


Объединим полученные датафреймы

In [87]:
merged_data = pd.merge(duration_data.rename({'call_date': 'date'}, axis=1), messages_data.rename({'message_date': 'date'}, axis=1), on=['user_id', 'date', 'month'])

merged_data = pd.merge(merged_data, internet_traff_data.rename({'session_date': 'date'}, axis=1), on=['user_id', 'date', 'month'])

merged_data = pd.merge(merged_data, users[['user_id', 'tariff']], on='user_id').rename(columns={'date': 'month_number'})

merged_data.head()

,user_id,month_number,rounded_duration,month,messages_count,mb_used,rounded_gb_used,tariff
0,1000,5,159,Май,22,2254.0,3,ultra
1,1000,6,172,Июнь,60,23232.0,23,ultra
2,1000,7,340,Июль,75,14008.0,14,ultra
3,1000,8,408,Август,81,14056.0,14,ultra
4,1000,9,466,Сентябрь,57,14568.0,15,ultra


Посчитаем помесячную выручку с каждого пользователя.
Для этого вычтем бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика,
а остаток умножим на значение из тарифного плана и прибавим абонентскую плату, соответствующую тарифному плану.

In [88]:
def calculate_month_revenue(row: pd.Series) -> int:
    # получаем тарифные лимиты
    limits = tariffs.query("tariff_name == @row['tariff']")
    messages_limit = int(limits["messages_included"])
    mb_limit = int(limits["mb_per_month_included"])
    minutes_limit = int(limits["minutes_included"])
    month_payment = int(limits["rub_monthly_fee"])
    payment_per_gb = int(limits["rub_per_gb"])
    payment_per_message = int(limits["rub_per_message"])
    payment_per_minute = int(limits["rub_per_minute"])

    # прибавим к стоимости тарифа плату за перерасход смс
    if row["messages_count"] > messages_limit:
        month_payment += (row["messages_count"] - messages_limit) * payment_per_message

    # прибавим к стоимости тарифа плату за перерасход гб интернетов
    if row["rounded_gb_used"] > (mb_limit / 1024):
        month_payment += ceil(row["rounded_gb_used"] - mb_limit / 1024) * payment_per_gb

    # прибавим к стоимости тарифа плату за перерасход минут
    if row["rounded_duration"] > minutes_limit:
        month_payment += (row["rounded_duration"] - minutes_limit) * payment_per_minute

    return month_payment

# посчитаем месячную выручку с каждого пользователя
merged_data["month_revenue"] = merged_data.apply(calculate_month_revenue, axis=1)

merged_data.head()

,user_id,month_number,rounded_duration,month,messages_count,mb_used,rounded_gb_used,tariff,month_revenue
0,1000,5,159,Май,22,2254.0,3,ultra,1950
1,1000,6,172,Июнь,60,23232.0,23,ultra,1950
2,1000,7,340,Июль,75,14008.0,14,ultra,1950
3,1000,8,408,Август,81,14056.0,14,ultra,1950
4,1000,9,466,Сентябрь,57,14568.0,15,ultra,1950


## ШАГ 3. Анализ данных


Посмотрим, кто является нашими пользователями. Из каких они городов и какой у них возраст.

In [202]:
# сгруппируем пользователей по городам
city_data = users.groupby(['city']).count().sort_values(by='user_id', ascending=False).reset_index()
# визуализируем данные
fig = px.bar(city_data,
             x='city',
             y='user_id',
             color='city',
             text_auto=True,
             title="Распреджеление пользователей по городам")

fig.update_layout(
    showlegend=True,
    xaxis_title="Город",
    yaxis_title="Количество пользователей"
)

fig.show()

Большинство пользователей в выборке из Москвы и СпБ.
Посмотрим какой у них возраст.

In [209]:
# сгруппируем пользователей по возрасту
age_data = users.groupby(['age']).count().sort_values(by='user_id', ascending=False).reset_index()
# визуализируем данные
fig = px.bar(age_data,
             x='age',
             y='user_id',
             color='age',
             text_auto=True,
             title="Распреджеление пользователей по возрасту")

fig.update_layout(
    showlegend=True,
    xaxis_title="Возраст",
    yaxis_title="Количество пользователей"
)

fig.show()

In [207]:
users['age'].mean()

46.588

В выборке пользователи от 18 до 75 лет, средний возраст 46,5 лет.

Посчитаем сколько в среднем минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц.
А также дисперсию и стандартное отклонение.

In [89]:
# словарь с ключами из названий колонок и значением с текстовым описанием для дальнейшего вывода
columns = {
    'rounded_duration': 'В среднем пользователю требуется _минут _дисперсия: _стандартное отклонение:',
    'messages_count': 'В среднем пользователю требуется _сообщений _дисперсия: _стандартное отклонение:',
    'rounded_gb_used': 'В среднем пользователю требуется _Гб _дисперсия: _стандартное отклонение:',
    'month_revenue': 'В среднем выручка с пользователя составляет _руб _дисперсия: _стандартное отклонение:'
}

# список тарифов
tariffs = ['ultra', 'smart']

# сгруппируем данные по тарифам и посчитаем средние значения
means = merged_data.groupby('tariff').mean().reset_index()

# для каждого тарифа
for tariff in tariffs:
    # разделитель
    display(f"{'=' * 20} {tariff} {'=' * 20}")
    # для каждой исследуемой колонки
    for column, text in zip(columns.keys(), columns.values()):
        # распарсим предопределенный для колонки текст с описанием
        text, type, var, sigma = text.split('_')
        # посчитаем дисперсию
        var_value = int(np.var(merged_data.query("tariff == @tariff")[column]))
        # посчитаем стандартное отклонение
        sigma_value = int(np.sqrt(var_value))
        # сформируем описательную строку для данных колонки
        display(f"{text}{int(means[means['tariff'] == tariff][column])} {type}, {var} {var_value}, {sigma} {sigma_value}")


'==================== ultra ===================='

'В среднем пользователю требуется 556 минут , дисперсия:  97129, стандартное отклонение: 311'

'В среднем пользователю требуется 61 сообщений , дисперсия:  2005, стандартное отклонение: 44'

'В среднем пользователю требуется 19 Гб , дисперсия:  87, стандартное отклонение: 9'

'В среднем выручка с пользователя составляет 2048 руб , дисперсия:  110320, стандартное отклонение: 332'

'==================== smart ===================='

'В среднем пользователю требуется 421 минут , дисперсия:  35217, стандартное отклонение: 187'

'В среднем пользователю требуется 38 сообщений , дисперсия:  718, стандартное отклонение: 26'

'В среднем пользователю требуется 16 Гб , дисперсия:  32, стандартное отклонение: 5'

'В среднем выручка с пользователя составляет 1289 руб , дисперсия:  676197, стандартное отклонение: 822'

Промежуточные выводы:
1. В тарифе ultra пользователи разговаривают на 24% дольше чем пользователи тарифа smart, в среднем на 135 минут.
2. В тарифе ultra пользователи отправляют на 38% больше сообщений в месяц чем пользователи тарифа smart, в среднем больше на 23 сообщения в месяц.
3. В тарифе ultra пользователи расходуют на 16% больше Гб интернета в месяц, чем пользователи тарифа smart, в среднем на 3 Гб в месяц.
4. Средняя выручка с пользователя тарифом ultra на 37% больше, чем пользователя тарифом smart, в среднем на 759 руб в месяц.
5. По правилу 3-х сигм 95% пользователей тарифа ultra укладываются в следующие показатели:
    - продолжительность вызова не более 933 минут
    - отправляют не более 132 сообщений в месяц
    - используют не более 27 Гб интернет трафика
    -
6. По правилу 3-х сигм 95% пользователей тарифа smart укладываются в следующие показатели:
    - продолжительность вызова не более 561 минут
    - отправляют не более 78 сообщений в месяц
    - используют не более 15 Гб интернет трафика

Посмотрим на редкие показатели поведения пользователей для каждого тарифа.

In [90]:
for tariff in tariffs:
    display(f"{'=' * 20} {tariff} {'=' * 20}")
    display(merged_data.query("tariff == @tariff").quantile([0.01, 0.1, 0.8, 0.9, 0.99])[['rounded_duration', 'messages_count', 'mb_used', 'rounded_gb_used', 'month_revenue']])

'==================== ultra ===================='

,rounded_duration,messages_count,mb_used,rounded_gb_used,month_revenue
0.01,13.53,2.00,1764.84,2.0,1950.0
0.10,142.10,10.00,6676.00,7.0,1950.0
0.80,816.00,98.00,28172.80,28.0,1950.0
0.90,961.90,130.00,32444.80,32.0,2250.0
0.99,1315.90,179.47,42287.04,42.0,3750.0


'==================== smart ===================='

,rounded_duration,messages_count,mb_used,rounded_gb_used,month_revenue
0.01,22.0,1.0,1844.28,2.0,550.00
0.10,174.0,8.0,8651.20,9.0,550.00
0.80,576.2,61.2,20896.00,21.0,1950.00
0.90,657.6,77.0,23216.00,23.0,2412.40
0.99,859.0,109.0,30030.72,30.0,3773.86


У нас есть как минимум 20% пользователей тарифа smart, которые не вписываются в его лимиты и переплачивают. Логично было бы предложить им перейти на тариф ultra.

Построим boxplot для визуализации средних значений колонок по месяцам

In [94]:
# словарь с ключами из названий колонок и описанием для визуализации
columns = {
    'rounded_duration': 'Время звонка (мин)',
    'messages_count': 'Количество сообщений',
    'rounded_gb_used': 'Использование интернета (гб)'
}

# построим boxplot для каждой колонки датафрейма merged_data
for column, title in zip(columns.keys(), columns.values()):
    fig = px.box(merged_data.sort_values(by='month_number'), x="month", y=column, color="tariff")
    fig.update_layout(
        showlegend=True,
        xaxis_title="Месяц",
        yaxis_title=title
    )

    fig.show()

Построим сводную таблицу выручки пользователей по месяцам и отсортируем значения по убыванию средней выручки.

In [95]:
merged_data.pivot_table(index=['tariff', 'month'], values=['month_revenue'], aggfunc=['sum', 'mean']).sort_values(by=('mean', 'month_revenue'), ascending=False)

sum          mean
                month_revenue month_revenue
tariff month                               
ultra  Декабрь         234600   2113.513514
       Ноябрь          222450   2078.971963
       Июль            143250   2076.086957
       Октябрь         200550   2067.525773
       Август          168600   2031.325301
       Июнь            127200   2019.047619
       Май              96900   2018.750000
       Март             50400   2016.000000
       Сентябрь        180450   2005.000000
       Февраль          27900   1992.857143
       Апрель           74700   1965.789474
       Январь           13650   1950.000000
smart  Декабрь         441541   1543.849650
       Октябрь         345600   1366.007905
       Август          274161   1357.232673
       Ноябрь          361549   1344.048327
       Сентябрь        279192   1257.621622
       Июль            213350   1226.149425
       Май             148867   1220.221311
       Июнь            170699   1210.631206
       Март             77453   1122.507246
       Апрель           93560    974.583333
       Февраль          41201    824.020000
       Январь           22675    731.451613

Промежуточные выводы:
1. В среднем пользователи двух тарифов больше всего переплачивают в декабре.
2. Пользователи тарифа ultra вообще не переплачивают за тариф в январе, а пользователи тарифа smart в среднем всегда переплачивают за пользование тарифом, но больше всего в декабре.
3. Средняя выручка по всем месяцам выше у тарифа ultra.

Скорее всего декабрьские переплаты связаны с новогодними праздниками и поздравлениями родных, близких и друзей, которых нет рядом в этот момент.

Построим аналогичную предыдущей сводной таблицы, но с сортировкой по суммарной выручке в месяц.

In [96]:
merged_data.pivot_table(index=['tariff', 'month'], values=['month_revenue'], aggfunc=['sum', 'mean']).sort_values(by=('sum', 'month_revenue'), ascending=False)

sum          mean
                month_revenue month_revenue
tariff month                               
smart  Декабрь         441541   1543.849650
       Ноябрь          361549   1344.048327
       Октябрь         345600   1366.007905
       Сентябрь        279192   1257.621622
       Август          274161   1357.232673
ultra  Декабрь         234600   2113.513514
       Ноябрь          222450   2078.971963
smart  Июль            213350   1226.149425
ultra  Октябрь         200550   2067.525773
       Сентябрь        180450   2005.000000
smart  Июнь            170699   1210.631206
ultra  Август          168600   2031.325301
smart  Май             148867   1220.221311
ultra  Июль            143250   2076.086957
       Июнь            127200   2019.047619
       Май              96900   2018.750000
smart  Апрель           93560    974.583333
       Март             77453   1122.507246
ultra  Апрель           74700   1965.789474
       Март             50400   2016.000000
smart  Февраль          41201    824.020000
ultra  Февраль          27900   1992.857143
smart  Январь           22675    731.451613
ultra  Январь           13650   1950.000000

Суммарная выручка в тарифе smart в конце года значительно больше, чем у тарифа ultra. Проверим, не связано ли это с количеством пользователей.

In [97]:
merged_data.pivot_table(index=['tariff', 'month'], values=['user_id'], aggfunc=['count']).sort_values(by=('count', 'user_id'), ascending=False)

count
                user_id
tariff month           
smart  Декабрь      286
       Ноябрь       269
       Октябрь      253
       Сентябрь     222
       Август       202
       Июль         174
       Июнь         141
       Май          122
ultra  Декабрь      111
       Ноябрь       107
       Октябрь       97
smart  Апрель        96
ultra  Сентябрь      90
       Август        83
       Июль          69
smart  Март          69
ultra  Июнь          63
smart  Февраль       50
ultra  Май           48
       Апрель        38
smart  Январь        31
ultra  Март          25
       Февраль       14
       Январь         7

Визуализируем данные по выручке

In [100]:
month_revenue = merged_data.groupby(['tariff', 'month']).sum().reset_index().sort_values(by='month_number')

fig = px.bar(month_revenue,
             x='month',
             y='month_revenue',
             color='tariff',
             text_auto=True,
             title="Суммарная выручка в месяц")

fig.update_layout(
    showlegend=True,
    xaxis_title="Месяц",
    yaxis_title="Выручка (руб)"
)

fig.show()

Суммарная выручка каждый месяц растет! Выглядит будто тариф смарт имеет более значительный прирост в месяц, чем ultra. Проверим это построив график процентного соотношения выручки по тарифам в месяц.

In [108]:
month_revenue = merged_data.groupby(['tariff', 'month']).sum().sort_values(by='month_number')
# создадим колонку percent со значением 0
month_revenue['percent'] = 0
# посчитаем процент выручки тарифа smart в месяц
percent_data = month_revenue.loc['smart']['month_revenue'] / (month_revenue.loc['smart']['month_revenue'] + month_revenue.loc['ultra']['month_revenue']) * 100
# добавим данные в датафрейм
month_revenue.loc['smart', 'percent'] = list(percent_data)
# посчитаем процент выручки тарифа ultra в месяц
percent_data = month_revenue.loc['ultra']['month_revenue'] / (month_revenue.loc['smart']['month_revenue'] + month_revenue.loc['ultra']['month_revenue']) * 100
# добавим данные в датафрейм
month_revenue.loc['ultra', 'percent'] = list(percent_data)
# сбросим индексы
percents = month_revenue.reset_index().sort_values(by='month_number')
# визуализируем данные о процентном соотношении выручки тарифов в месяц
fig = px.bar(percents,
             x='month',
             y='percent',
             color='tariff',
             text_auto=True,
             title="Процентное соотношение выручки тарифов по месяцам")

fig.update_layout(
    showlegend=True,
    xaxis_title="Месяц",
    yaxis_title="Процент выручки"
)

fig.show()

Процент выручки по тарифу не сильно менялся из месяца в месяц, он был более-менее стабилен. На момент конца 2018 года smart приносит ~65% выручки в месяц, а ultra ~35%.

Увеличение суммарной выручки каждый месяц может быть связано с количеством пользователей, проверим это.
Визуализируем данные по количеству пользователей в месяц.

In [101]:
month_users_count = merged_data.groupby(['tariff', 'month']).count().reset_index().sort_values(by='month_number')

fig = px.bar(month_users_count,
             x='month',
             y='user_id',
             color='tariff',
             text_auto=True,
             title="Количество пользователей в месяц")

fig.update_layout(
    showlegend=True,
    xaxis_title="Месяц",
    yaxis_title="Количество пользователей"
)

fig.show()

Количество пользователей с каждым месяцем растет! Тариф smart пользуется бОльшей популярностью, чем ultra. Вероятно большинству не нужны большие лимиты в тарифе.
Визуализируем данные о процентном соотношении пользователей тарифов в месяц

In [105]:
# сгруппируем по тарифу и дате
percents = merged_data.groupby(['tariff', 'month']).count().sort_values(by='month_number')
# создадим колонку percent со значением 0
percents['percent'] = 0
# посчитаем процент пользователей тарифа smart в месяц
percent_data = percents.loc['smart']['user_id'] / (percents.loc['smart']['user_id'] + percents.loc['ultra']['user_id']) * 100
# добавим данные в датафрейм
percents.loc['smart', 'percent'] = list(percent_data)

# посчитаем процент пользователей тарифа ultra в месяц
percent_data = percents.loc['ultra']['user_id'] / (percents.loc['smart']['user_id'] + percents.loc['ultra']['user_id']) * 100
# добавим данные в датафрейм
percents.loc['ultra', 'percent'] = list(percent_data)
# сбросим индексы
percents = percents.reset_index().sort_values(by='month_number')
# визуализируем данные о процентном соотношении пользователей тарифов в месяц
fig = px.bar(percents,
             x='month',
             y='percent',
             color='tariff',
             text_auto=True,
             title="Процентное соотношение пользователей тарифов по месяцам")

fig.update_layout(
    showlegend=True,
    xaxis_title="Месяц",
    yaxis_title="Процент пользователей"
)

fig.show()

Промежуточные выводы:
1. Количество пользователей растет каждый месяц.
2. Тариф smart в сумме приносит больше денег, чем ultra (на момент конца 2018 года smart: ~65% выручки, ultra: ~35% выручки)
3. Тариф smart всегда был более популярен, чем ultra (на момент конца 2018 года smart: ~72% пользователей, ultra: ~28%)

## Шаг 4. Проверим гипотезы

Проверим гипотезу, что средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются.
Т.о. нулевая гипотеза звучит следующим образом: средняя выручка пользователей тарифов «Ультра» и «Смарт» равны (k1 = k2).
Альтернативная гипотеза: средняя выручка пользователей тарифа «Ультра» не равна средней выручке пользователей тарифа «Смарт» (k1 != k2).

Т.к. перед нами стоит задача проверки двусторонней гипотезы о средних двух генеральных совокупностей - применим метод ttest.

In [145]:
# выберем данные о выручке пользователей по каждому тарифу
ultra = merged_data.query("tariff == 'ultra'")['month_revenue']
smart = merged_data.query("tariff == 'smart'")['month_revenue']

# зададим пороговое значение
alpha = .01
# применим метод для проверки гипотезы о равенстве среднего двух генеральных совокупностей
results = st.ttest_ind(smart, ultra)

if results.pvalue < alpha:
    print("Опровергаем нулевую гипотезу. Средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются.")
else:
    print("Подтверждаем нулевую гипотезу. Средняя выручка пользователей тарифов «Ультра» и «Смарт» равны.")


Опровергаем нулевую гипотезу. Средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются.


Проверим следующую гипотезу: средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов.

Т.о. нулевая гипотеза звучит следующим образом: средняя выручка пользователей из Москвы и из других городов равны (k1 = k2).
Альтернативная гипотеза: средняя выручка пользователей из Москвы не равна средней выручке пользователей из других городов (k1 != k2).
Т.к. перед нами снова стоит задача проверки двусторонней гипотезы о средних двух генеральных совокупностей - применим метод ttest.

In [157]:
# добавим данные о принадлежности пользователей к городам
citys = pd.merge(merged_data, users[['user_id', 'city']], on='user_id')

# выберем данные о выручке пользователей по каждому тарифу
moscow = citys.query("city == 'москва'")['month_revenue']
other_citys = citys.query("city != 'москва'")['month_revenue']

# зададим пороговое значение
alpha = .01
# применим метод для проверки гипотезы о равенстве среднего двух генеральных совокупностей
results = st.ttest_ind(moscow, other_citys)

if results.pvalue < alpha:
    print("Опровергаем нулевую гипотезу. Средняя выручка пользователей из Москвы и других городов различаются.")
else:
    print("Подтверждаем нулевую гипотезу. Средняя выручка пользователей из Москвы и других городов равны.")

Подтверждаем нулевую гипотезу. Средняя выручка пользователей из Москвы и других городов равны.


## Шаг 5. Общие выводы.

В данном исследовании мы рассмотрели данные 500 пользователей федерального оператора сотовой связи «Мегалайн» и можем сделать следующие выводы:
1. В выборке пользователи из разных городов РФ, но большинство пользователей из Москвы и Спб, возраст всех пользователей от 18 до 75, средний возраст аудитории 46,5 лет.
2. Каждый месяц выручка оператора связи по двум тарифам растет.
3. Пользователи тарифа smart в среднем всегда переплачивают за пользование тарифом.
4. Средняя выручка с пользователя тарифом ultra на 37% больше, чем средняя выручка с пользователя тарифом smart.
5. У нас есть как минимум 20% пользователей тарифа smart, которые не вписываются в его лимиты и переплачивают. Логично было бы предложить им перейти на тариф ultra.
6. Тариф smart на момент конца 2018 года генерирует ~65% выручки и им пользуется ~72% пользователей, а ultra генерирует ~35% выручки и им пользуется ~28% пользователей.
7. Средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются.
8. Средняя выручка пользователей из Москвы и других городов равны.